# 1.1 Import Needed Libraries

In [34]:
import pandas as pd
import numpy as np
import os

from copy import deepcopy
from random import random, sample, choice, randint

# 1.2 Import Our Dataset

In [35]:
data_dir='/Users/minguini/Documents/GitHub/CIFO_24_25'

In [36]:
df = pd.read_csv(path.join(data_dir, 'players(in).csv'), index_col=0)
df.head()

,Name,Position,Skill,Salary (€M)
0,Alex Carter,GK,85,90
1,Jordan Smith,GK,88,100
2,Ryan Mitchell,GK,83,85
3,Chris Thompson,GK,80,80
4,Blake Henderson,GK,87,95


In [37]:
players = df.to_dict(orient="records")
players

[{'Name': 'Alex Carter', 'Position': 'GK', 'Skill': 85, 'Salary (€M)': 90},
 {'Name': 'Jordan Smith', 'Position': 'GK', 'Skill': 88, 'Salary (€M)': 100},
 {'Name': 'Ryan Mitchell', 'Position': 'GK', 'Skill': 83, 'Salary (€M)': 85},
 {'Name': 'Chris Thompson', 'Position': 'GK', 'Skill': 80, 'Salary (€M)': 80},
 {'Name': 'Blake Henderson', 'Position': 'GK', 'Skill': 87, 'Salary (€M)': 95},
 {'Name': 'Daniel Foster', 'Position': 'DEF', 'Skill': 90, 'Salary (€M)': 110},
 {'Name': 'Lucas Bennett', 'Position': 'DEF', 'Skill': 85, 'Salary (€M)': 90},
 {'Name': 'Owen Parker', 'Position': 'DEF', 'Skill': 88, 'Salary (€M)': 100},
 {'Name': 'Ethan Howard', 'Position': 'DEF', 'Skill': 80, 'Salary (€M)': 70},
 {'Name': 'Mason Reed', 'Position': 'DEF', 'Skill': 82, 'Salary (€M)': 75},
 {'Name': 'Logan Brooks', 'Position': 'DEF', 'Skill': 86, 'Salary (€M)': 95},
 {'Name': 'Caleb Fisher', 'Position': 'DEF', 'Skill': 84, 'Salary (€M)': 85},
 {'Name': 'Nathan Wright', 'Position': 'MID', 'Skill': 92, 'Sa

In [38]:
def players_by_position(players):
    gk = []
    defenders = []
    midfielders = []
    forwards = []
    for i in players:
        if i["Position"] == "GK":
            gk.append(i)
        elif i["Position"] == "DEF":
            defenders.append(i)
        elif i["Position"] == "MID":
            midfielders.append(i)
        elif i["Position"] == "FWD":
            forwards.append(i)

    return gk, defenders, midfielders, forwards

In [39]:
gk, defenders, midfielders, forwards = players_by_position(players)
gk

[{'Name': 'Alex Carter', 'Position': 'GK', 'Skill': 85, 'Salary (€M)': 90},
 {'Name': 'Jordan Smith', 'Position': 'GK', 'Skill': 88, 'Salary (€M)': 100},
 {'Name': 'Ryan Mitchell', 'Position': 'GK', 'Skill': 83, 'Salary (€M)': 85},
 {'Name': 'Chris Thompson', 'Position': 'GK', 'Skill': 80, 'Salary (€M)': 80},
 {'Name': 'Blake Henderson', 'Position': 'GK', 'Skill': 87, 'Salary (€M)': 95}]

# 2. Fitness Function

In [43]:
class League():
    def __init__(self, name, players):
        self.name = name
        self.players = players
        self.gk = []
        self.defenders = []
        self.midfielders = []
        self.forwards = []
        self.total_points = 0
        self.total_cost = 0

        for player in players:
            self.add_player(player)

    def random_teams(players, num_teams = 5, budget=100):
        teams = []
        positions = {'Gk': [], 'DEF': [], 'MID': [], 'FWD': []}

        for player in players:
            positions[player['Position']].append(player)
        
        #Generate teams
        for i in range(num_teams):
            while True:
                team = [
                    random.choice(positions['Gk']),
                    random.choice(positions['DEF'], k=2),
                    random.choice(positions['MID'], k=2),
                    random.choice(positions['FWD'], k=2),
                ]
                #Check the budget and unique players
                if len(set(team)) == len(team) and sum(player['Cost'] for player in team) <= budget:
                    teams.append(team)
                    break
        return teams
        

    def add_player(self, player):
        if player["Position"] == "GK":
            self.gk.append(player)
        elif player["Position"] == "DEF":
            self.defenders.append(player)
        elif player["Position"] == "MID":
            self.midfielders.append(player)
        elif player["Position"] == "FWD":
            self.forwards.append(player)

    def valid_formation(self):
        return (len(self.gk) == 1 and
                len(self.defenders) == 2 and
                len(self.midfielders) == 2 and
                len(self.forwards) == 2)

    def valid_budget(self, budget):
        return self.calculate_total_cost() <= budget
    
    def calculate_total_skill(self):
        self.total_skill = sum([player["Skill"] for player in self.players])
        return self.total_skill
    
    def calculate_total_cost(self):
        self.total_cost = sum([player["Cost"] for player in self.players])
        return self.total_cost
    
    def fitness(self, budget):
        if not self.valid_budget(budget):
            return -100000
        else:
            cost_penalty = max(0, self.calculate_total_cost() - 100) * 5
            return self.calculate_total_skill() - cost_penalty
    

    # Magic methods
    def __str__(self):
        return f"Team: {self.name}, Players: {len(self.players)}, Total Points: {self.total_points}, Total Cost: {self.total_cost}"
    def __repr__(self):
        return f"Team({self.name}, {self.players})"
    def __len__(self):
        return len(self.players)
    def __getitem__(self, index):
        return self.players[index]
    def __setitem__(self, index, value):
        self.players[index] = value
    def __delitem__(self, index):
        del self.players[index]
    def __contains__(self, item):
        return item in self.players
    def __iter__(self):
        return iter(self.players)

In [41]:
solution = Team()

print(solution)
print(solution.random_initial_representation())
print(solution.fitness())

TypeError: Team.__init__() missing 2 required positional arguments: 'name' and 'players'

In [28]:
#Fitness function

def fitness(team):
    if not team.valid_formation():
        return -100000  # Hard constraint penalty
    
    cost_penalty = max(0, team.calculate_total_cost() - 100) * 5
    return team.calculate_total_points() - cost_penalty

In [ ]:
def initialize_population(player_pool, population_size=100, max_budget=100):
    population = []
    position_groups = {
        'GK': [p for p in player_pool if p['Position'] == 'GK'],
        'DEF': [p for p in player_pool if p['Position'] == 'DEF'],
        'MID': [p for p in player_pool if p['Position'] == 'MID'],
        'FWD': [p for p in player_pool if p['Position'] == 'FWD']
    }
    
    for _ in range(population_size):
        while True:
            team_players = [
                random.choice(position_groups['GK']),
                *random.sample(position_groups['DEF'], k=2),
                *random.sample(position_groups['MID'], k=2),
                *random.sample(position_groups['FWD'], k=2)
            ]
            
            # Add optional players (up to 11 total)
            remaining_spots = 11 - len(team_players)
            for _ in range(remaining_spots):
                position = random.choice(['DEF', 'MID', 'FWD'])
                team_players.append(random.choice(position_groups[position]))
                
            # Check budget constraint
            total_cost = sum(p['Cost'] for p in team_players)
            if total_cost <= max_budget:
                population.append(Team(f"Team_{len(population)+1}", team_players))
                break
                
    return population


In [30]:
solution = Team("Best Team", [])
print(solution)

Team: Best Team, Players: 0, Total Points: 0, Total Cost: 0


In [8]:
#vefify if team has the right number of players, and the right number of players for each position
def valid_team(team):
    n_gk = 0 #number of gk per team
    n_def = 0
    n_mid = 0
    n_forwards = 0

    for i in team:
        if i["Position"] == "GK":
            n_gk += 1
        elif i["Position"] == "DEF":
            n_def += 1
        elif i["Position"] == "MID":
            n_mid += 1
        elif i["Position"] == "FWD":
            n_forwards += 1

    if len(team) != 7:
        return False
    if n_gk != 1:
        return False
    if n_def != 2:
        return False
    if n_mid != 2:
        return False
    if n_forwards != 2:
        return False

    return True  

#function that sums all salaries of each player and returns the total
def total_cost(team):
    total_cost = 0

    for i in team:
        total_cost += + i["Salary (€M)"]
    return total_cost


#verify if each player is assigned to exactly one team
def has_repeated_players(solution):

    players = []
    for team in solution: #5 teams
        for player in team: #7 players for each team
            if player['Name'] in players:
                return True
            players.append(player['Name'])
    
    return False

In [9]:
#example of solutions to check 
solution1 = [
    [  # Team 1
        {'Name': 'Alex Carter', 'Position': 'GK', 'Skill': 85, 'Salary (€M)': 90},
        {'Name': 'Daniel Foster', 'Position': 'DEF', 'Skill': 90, 'Salary (€M)': 110},
        {'Name': 'Lucas Bennett', 'Position': 'DEF', 'Skill': 85, 'Salary (€M)': 90},
        {'Name': 'Nathan Wright', 'Position': 'MID', 'Skill': 92, 'Salary (€M)': 120},
        {'Name': 'Connor Hayes', 'Position': 'MID', 'Skill': 89, 'Salary (€M)': 105},
        {'Name': 'Sebastian Perry', 'Position': 'FWD', 'Skill': 95, 'Salary (€M)': 150},
        {'Name': 'Tyler Jenkins', 'Position': 'FWD', 'Skill': 80, 'Salary (€M)': 70}
        ],
    [  # Team 2
        {'Name': 'Jordan Smith', 'Position': 'GK', 'Skill': 88, 'Salary (€M)': 100},
        {'Name': 'Owen Parker', 'Position': 'DEF', 'Skill': 88, 'Salary (€M)': 100},
        {'Name': 'Ethan Howard', 'Position': 'DEF', 'Skill': 80, 'Salary (€M)': 70},
        {'Name': 'Dylan Morgan', 'Position': 'MID', 'Skill': 91, 'Salary (€M)': 115},
        {'Name': 'Hunter Cooper', 'Position': 'MID', 'Skill': 83, 'Salary (€M)': 85},
        {'Name': 'Xavier Bryant', 'Position': 'FWD', 'Skill': 90, 'Salary (€M)': 120},
        {'Name': 'Adrian Collins', 'Position': 'FWD', 'Skill': 85, 'Salary (€M)': 90}
    ]
]

#example of solution with repeated name
solution2 = [
    [  # Team 1
        {'Name': 'Alex Carter', 'Position': 'GK', 'Skill': 85, 'Salary (€M)': 90},
        {'Name': 'Alex Carter', 'Position': 'GK', 'Skill': 85, 'Salary (€M)': 90}, #repeated
        {'Name': 'Lucas Bennett', 'Position': 'DEF', 'Skill': 85, 'Salary (€M)': 90},
        {'Name': 'Nathan Wright', 'Position': 'MID', 'Skill': 92, 'Salary (€M)': 120},
        {'Name': 'Connor Hayes', 'Position': 'MID', 'Skill': 89, 'Salary (€M)': 105},
        {'Name': 'Sebastian Perry', 'Position': 'FWD', 'Skill': 95, 'Salary (€M)': 150},
        {'Name': 'Tyler Jenkins', 'Position': 'FWD', 'Skill': 80, 'Salary (€M)': 70}
    ],
    [  # Team 2
        {'Name': 'Jordan Smith', 'Position': 'GK', 'Skill': 88, 'Salary (€M)': 100},
        {'Name': 'Owen Parker', 'Position': 'DEF', 'Skill': 88, 'Salary (€M)': 100},
        {'Name': 'Ethan Howard', 'Position': 'DEF', 'Skill': 80, 'Salary (€M)': 70},
        {'Name': 'Dylan Morgan', 'Position': 'MID', 'Skill': 91, 'Salary (€M)': 115},
        {'Name': 'Hunter Cooper', 'Position': 'MID', 'Skill': 83, 'Salary (€M)': 85},
        {'Name': 'Xavier Bryant', 'Position': 'FWD', 'Skill': 90, 'Salary (€M)': 120},
        {'Name': 'Adrian Collins', 'Position': 'FWD', 'Skill': 85, 'Salary (€M)': 90}
    ]
]

#example of solution with 6 players
solution3 = [
    [  # Team 1
        {'Name': 'Alex Carter', 'Position': 'GK', 'Skill': 85, 'Salary (€M)': 90},
        {'Name': 'Daniel Foster', 'Position': 'DEF', 'Skill': 90, 'Salary (€M)': 110},
        {'Name': 'Lucas Bennett', 'Position': 'DEF', 'Skill': 85, 'Salary (€M)': 90},
        {'Name': 'Nathan Wright', 'Position': 'MID', 'Skill': 92, 'Salary (€M)': 120},
        {'Name': 'Connor Hayes', 'Position': 'MID', 'Skill': 89, 'Salary (€M)': 105},
        {'Name': 'Sebastian Perry', 'Position': 'FWD', 'Skill': 95, 'Salary (€M)': 150},
        {'Name': 'Tyler Jenkins', 'Position': 'FWD', 'Skill': 80, 'Salary (€M)': 70}
    ],
    [  # Team 2
        {'Name': 'Jordan Smith', 'Position': 'GK', 'Skill': 88, 'Salary (€M)': 100},
        {'Name': 'Owen Parker', 'Position': 'DEF', 'Skill': 88, 'Salary (€M)': 100},
        {'Name': 'Ethan Howard', 'Position': 'DEF', 'Skill': 80, 'Salary (€M)': 70},
        {'Name': 'Dylan Morgan', 'Position': 'MID', 'Skill': 91, 'Salary (€M)': 115},
        {'Name': 'Hunter Cooper', 'Position': 'MID', 'Skill': 83, 'Salary (€M)': 85},
        {'Name': 'Xavier Bryant', 'Position': 'FWD', 'Skill': 90, 'Salary (€M)': 120}
        ]
]

In [ ]:
print("Has repeated players:")
print("Solution 1:", has_repeated_players(solution1))
print("Solution 2:", has_repeated_players(solution2))
print("Solution 3:", has_repeated_players(solution3))

print("\nTotal cost solution 1:")
print("Team 1:", total_cost(solution1[0]), "M €")
print("Team 2:", total_cost(solution1[1]), "M €")

print("\nSolution 2 has valid teams:")
print("Team 1:", valid_team(solution2[0]))
print("Team 2:", valid_team(solution2[1]))

has repeated players:
Solution 1: False
Solution 2: True
Solution 3: False

Total cost solution 1:
Team 1: 735 M €
Team 2: 680 M €

Solution 2 has valid teams:
Team 1: False
Team 2: True


In [ ]:
class Team:
    def __init__(self, name, ):
    